# Make your own hemodynamic response function

In [1]:
# Import array and plotting libraries.
import numpy as np
import matplotlib.pyplot as plt
# Print arrays to 4 decimal places
np.set_printoptions(precision=4, suppress=True)

# Nibabel and course libraries.
import nibabel as nib
import nipraxis

The file `mt_hrf_estimates.txt` contains the estimated FMRI signal from voxels
in the MT motion area at 0, 2, 4, …, 28 seconds after a brief moving visual
stimulus (see:
[http://nipy.org/nitime/examples/event_related_fmri.html](http://nipy.org/nitime/examples/event_related_fmri.html)).

Here are the first four rows. The numbers are in exponential floating
point format:

```
1.394086932967517900e-01
3.938585701431884800e-01
5.012927230566770476e-01
5.676763716149294536e-01
```

Read the values from the file into an array `mt_hrf_estimates`. Make a new
array `mt_hrf_times` with the times of acquisition (0, 2, …). Plot them
together to see the HRF estimate at these times:

In [2]:
#- Load the estimated values from the text file into an array
#- Make an array of corresponding times
mt_hrf_estimates = np.loadtxt('mt_hrf_estimates.txt')
mt_hrf_times = np.arange(0, 30, 2)
# Plot signal by time
plt.plot(mt_hrf_times, mt_hrf_estimates)

In [3]:
assert len(mt_hrf_times) == len(mt_hrf_estimates)
assert np.all(mt_hrf_times[:3] == [0, 2, 4])
assert mt_hrf_times[-1] == 28

We want to make a *hemodynamic response function* that matches this shape.

Our function will accept an array that gives the times we want to calculate
the HRF for, and returns the values of the HRF for those times. We will assume
that the true HRF starts at zero, and gets to zero sometime around 30 seconds.

Like SPM, I’m going to try using the sum of two [gamma
distribution](https://en.wikipedia.org/wiki/Gamma_distribution)
probability density functions.

In [4]:
#: import the gamma density function
from scipy.stats import gamma

Here’s my first shot:

In [5]:
#: my attempt - you can do better than this
def not_great_hrf(times):
    """ Return values for HRF at given times """
    # Gamma pdf for the peak
    peak_values = gamma.pdf(times, 6)
    # Gamma pdf for the undershoot
    undershoot_values = gamma.pdf(times, 12)
    # Combine them
    values = peak_values - 0.35 * undershoot_values
    # Scale max to 0.6
    return values / np.max(values) * 0.6

In [6]:
#: plot the data against my estimate
plt.plot(mt_hrf_times, not_great_hrf(mt_hrf_times), label='not_great_hrf')
plt.plot(mt_hrf_times, mt_hrf_estimates, label='mt_hrf_estimates')
plt.legend()

Now see if you can make a better function by playing with the Gamma
distribution PDF parameter, and the mix of the two gamma distribution
functions. Call your function `mt_hrf`

In [7]:
#- Your "mt_hrf" function here
def mt_hrf(times):
    """ Return values for HRF at given times """
    # Gamma pdf for the peak
    peak_values = gamma.pdf(times, 7)
    # Gamma pdf for the undershoot
    undershoot_values = gamma.pdf(times, 20)
    # Combine them
    values = peak_values - undershoot_values
    # Scale max to 0.6
    return values / np.max(values) * 0.6

In [8]:
#- Plot your function against the mt_hrf_estimates data to test
plt.plot(mt_hrf_times, mt_hrf(mt_hrf_times), label='mt_hrf')
plt.plot(mt_hrf_times, mt_hrf_estimates, label='mt_hrf_estimates')
plt.legend()

For extra points - other than looking at these plots, how would you
convince me your function is better than mine?

In [9]:
#- Evidence that your function is better than mine?
#- Your code below, to persuade me.
np.corrcoef(mt_hrf_times, not_great_hrf(mt_hrf_times))
np.corrcoef(mt_hrf_times, mt_hrf(mt_hrf_times))

## Modeling

We are going to be analyzing the data for the 4D image
`ds114_sub009_t2r1.nii` again.

In [10]:
# Fetch the data file.
data_fname = nipraxis.fetch_file('ds114_sub009_t2r1.nii')
# Show the file name of the fetched data.
img = nib.load(data_fname)
# Load data, drop first (bad) volume.
data = img.get_fdata()[..., 1:]
data.shape

In [11]:
# Fetch, load the condition file
from nipraxis.stimuli import events2neural

cond_fname = nipraxis.fetch_file('ds114_sub009_t2r1_cond.txt')
TR = 2.5  # time between volumes
n_trs = img.shape[-1]  # The original number of TRs
neural_prediction = events2neural(cond_fname, TR, n_trs)
# Drop value corresponding to first bad volume, dropped above.
neural_prediction = neural_prediction[1:]
plt.plot(neural_prediction)
plt.ylim(0, 1.2)

For now, we’re going to play with data for a single voxel.

In the *First go at brain activation* exercise, we subtracted the rest scans
from the task scans, something like this:

In [12]:
#: subtracting rest from task scans
task_scans = data[..., neural_prediction == 1]
rest_scans = data[..., neural_prediction == 0]
difference = np.mean(task_scans, axis=-1) - np.mean(rest_scans, axis=-1)

In [13]:
#: showing slice 14 from the difference image
plt.imshow(difference[:, :, 14], cmap='gray')

It looks like there’s a voxel that is greater for activation than rest at
about (i, j, k) == (45, 43, 14).

Get and plot the values for this voxel position, for every volume in the 4D
data (not including the first volume). You can do it with a loop, but slicing
is much nicer.

In [14]:
#- Get the values for (i, j, k) = (45, 43, 14) and every volume.
voxel_values = data[45, 43, 14, :]
# Plot the values (voxel time course).
plt.plot(voxel_values)

In [15]:
assert len(voxel_values) == 172

Correlate the predicted neural time series with the voxel time course:

In [16]:
#- Correlation of predicted neural time course with voxel signal time
#- course
correlation = np.corrcoef(neural_prediction, voxel_values)[0, 1]
# Show the result.
correlation

In [17]:
# Should be scalar (single value).  Did you select the value
# from the correlation array?
assert correlation.size == 1
assert np.round(correlation, 3) == 0.312

Now we will do a predicted hemodynamic time course using convolution.

Next we need to get the HRF vector to convolve with.

Remember we have defined the HRF as a function of time, not TRs.

For our convolution, we need to *sample* the HRF at times corresponding the
start of the TRs.

So, we need to sample at times (0, 2.5, …)

Make a vector of times at which to sample the HRF. We want to sample every TR
up until (but not including) somewhere near 35 seconds (where the HRF should
have got close to zero again).

In [18]:
#- Make a vector of times at which to sample the HRF
hrf_times = np.arange(0, 35, TR)
# Show the result
hrf_times

In [19]:
assert np.allclose(hrf_times[:4], [0, 2.5, 5, 7.5])
assert np.allclose(hrf_times[-2:], [30.0, 32.5])
assert len(hrf_times) == 14

Sample your `mt_hrf` HRF function at these times and plot:

In [20]:
#- Sample HRF at given times
hrf_signal = mt_hrf(hrf_times)
#- Plot HRF samples against times
plt.plot(hrf_times, hrf_signal)

Convolve the predicted neural time course with the HRF samples:

In [21]:
#- Convolve predicted neural time course with HRF samples
hemodynamic_prediction = np.convolve(neural_prediction, hrf_signal)
# Show the first 10 values in the result
hemodynamic_prediction[:10]

The default output of convolve is longer than the input neural prediction
vector, by the length of the convolving vector (the HRF samples) minus 1.
Knock these last values off the result of the convolution to get a vector the
same length as the neural prediction:

In [22]:
#- Remove extra tail of values put there by the convolution
hemo_pred_short = hemodynamic_prediction[:len(neural_prediction)]

In [23]:
assert len(hemo_pred_short) == len(voxel_values)

Plot the convolved neural prediction, and then, on the same plot, plot the
unconvolved neural prediction.

In [24]:
#- Plot convolved neural prediction and unconvolved neural prediction
plt.plot(neural_prediction, label='unconvolved')
plt.plot(hemo_pred_short, label='convolved')
plt.legend()

Does the new convolved time course correlate better with the voxel time
course?

In [25]:
#- Correlation of the convolved time course with voxel time course
np.corrcoef(hemo_pred_short, voxel_values)

Plot the hemodynamic prediction against the actual signal (voxel values).  Investigate the `plt.` namespace for a useful function to do a scatterplot.

In [26]:
#- Scatterplot the hemodynamic prediction against the signal
plt.scatter(hemo_pred_short, voxel_values)
plt.xlabel('hemodynamic prediction')
plt.ylabel('voxel values')